In [244]:
# Importe de librerías
from sympy import *
from sympy.physics.mechanics import *
init_vprinting(pretty_print=True)
from sympy.solvers.ode.systems import dsolve_system

## Problema 2
Calcular el valor de las variables de control, de estado y coestado para los que se verifica el principio del máximo de Pontryagin en el siguiente problema.

\begin{equation}
    \begin{split}
        \min J(x) = & \int_0^2 (u^2 + 12u + 8x)\, dt \\
    \text{sujeto a : } & \,\, \dot{x} = 2x + 3u\\
    \text{con : } & \,\, x(0) = 5\\
    & \,\, 0 \leq u \leq 1
    \end{split}
\end{equation}

Inicialmente, se definen la variable independiente $t$, las funciones $x,u:\mathbb{R}^1 → \mathbb{R}$ y el integrando del funcional objetivo en forma de Lagrange $F = F(x,u,t) = u^2+12u+8x$. Además, se crean variables para las derivadas de $x$, y se declara la función de restricción $f = f(x,u,t) = 2x+3u$.

In [245]:
# Se declara la variable independiente t
t = Symbol('t')

# La variable de estado x y la variable de control u son funciones de t
x = Function('x')(t)
u = Function('u')(t)

# Se definen el integrando de la función objetivo y la función de restricción
F = u**2 + 12*u + 8*x
f = 2*x + 3*u

# Se declaran las derivadas de x(t)
dx = diff(x, t)
d2x = diff(dx, t)

# Se enseña F(x,u,t)
F

 2             
u  + 12⋅u + 8⋅x

Inicialmente, se define el hamiltoniano del sistema como:

\begin{align*}
H = H(x,u,λ,t) = F + λf
\end{align*}

donde $λ: \mathbb{R}^1 → \mathbb{R}$ es también una función de $t$.

In [246]:
# Se declara el multiplicador lambda del principio del máximo
lam = Function('lambda')(t)

# Se calcula el hamiltoniano como se enseña arriba
H = F + lam*f

# Se expone el hamiltoniano del sistema
H

                 2             
(3⋅u + 2⋅x)⋅λ + u  + 12⋅u + 8⋅x

Ahora, se emplea la condición $(1)$ del principio del máximo:

\begin{align*}
\dot{λ} = - \frac{\partial H}{∂ x}
\end{align*}

en conjunto con la condición inicial de la EDO anterior:

\begin{align*}
\lambda(2) = \frac{d}{dx}S(x^*(1)) = \frac{d}{dx}(0) = 0
\end{align*}

La condición de frontera es nula, dado que el problema está escrito en forma de Lagrange.

In [247]:
# Se define la primera derivada temporal de lam
dlam = diff(lam, t)

# Se establece la ecuación diferencial de la condición (1)
eq1 = Eq(dlam, -diff(H,x))
eq1

λ̇ = -2⋅λ - 8

In [248]:
# Se crea un diccionario con la condición inicial de la EDO λ(2) = 0
ics1 = {lam.subs(t,2) : 0}
display(ics1)

{λ(2): 0}

In [249]:
# Se resuelve la EDO anterior empleando el dsolve de Sympy, con las condiciones iniciales de ics1
sln_eq1 = dsolve_system([eq1], ics=ics1)
sln_lam = sln_eq1[0][0].doit()
sln_lam

/usr/local/lib/python3.10/dist-packages/sympy/simplify/radsimp.py:1131: SymPyDeprecationWarning: 

Using non-Expr arguments in Mul is deprecated (in this case, one of
the arguments has type 'Tuple').

If you really did intend to use a multiplication or addition operation with
this object, use the * or + operator instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#non-expr-args-deprecated
for details.

This has been deprecated since SymPy version 1.7. It
will be removed in a future version of SymPy.

  return Mul(*numer, evaluate=not exact), Mul(*denom, evaluate=not exact)


            4  -2⋅t
λ = -4 + 4⋅ℯ ⋅ℯ    

Así, con la condición $(1)$ del principio del máximo y resolviendo la EDO lineal, se halló $\lambda^*$, la función multiplicadora óptima como:

\begin{align*}
\lambda^*(t) = -4 + 4e^4e^{-2t}
\end{align*}

A continuación, se debe usar la condición $(2)$ del principio del máximo. Para ello hay que calcular:

\begin{align*}
\min_u H(x^*,u,\lambda^*,t)
\end{align*}

donde $u$ se encuentra irrestricta.

Como se trata de una función univariada que se debe optimizar sin restricciones, se debe cumplir que:

\begin{align*}
\frac{∂H}{∂u} = 0, \qquad \frac{∂^2H}{∂u^2} > 0
\end{align*}

que son las condiciones de primer y segundo orden para hallar un mínimo global.

In [250]:
# Se establece la condición de primer orden para minimuzar H
eq2 = Eq(diff(H,u), 0)
eq2

3⋅λ + 2⋅u + 12 = 0

In [251]:
# Despejando u de la ecuación anterior, se obtiene:
aux_u = solve([eq2], (u))
eq3 = Eq(u, aux_u[u])
eq3

      3⋅λ    
u = - ─── - 6
       2     

In [252]:
# Y reemplazando el lambda óptimo λ* hallado en la parte anterior se obtiene:
sln_u = eq3.subs({lam: sln_lam.rhs})
sln_u

        4  -2⋅t
u = -6⋅ℯ ⋅ℯ    

Usando $\frac{∂H}{∂u} = 0$, se obtuvo que la única función que satisface las condiciones de primer orden es:

\begin{align*}
u=-6e^4e^{-2t}
\end{align*}

veamos si esta corresponde a un máximo o a un mínimo derivando de nuevo el hamiltoniano.


In [253]:
# Derivando dos veces el hamiltoniano con respecto a u se obtiene:
diff(diff(H,u),u)

2

De esta forma $\frac{\partial^2 H}{\partial u^2} = 2 > 0$ (mínimo), por lo que $u^*(t) = -6e^4e^{-2t}$ es la función de control óptima $∀t ∈ [0,2]$.

Ahora, se debe usar la condición $(3)$ del principio del máximo que corresponde a resolver la EDO contemplada en la restricción del problema:

\begin{align*}
\dot{x}^* = f(x^*, u^*,t)
\end{align*}

sujeta a la condición inicial $x(0)=5$.

In [254]:
# Se establece la EDO de la restricción
eq4 = Eq(dx, f)
eq4

ẋ = 3⋅u + 2⋅x

In [255]:
# Se sustituye la u* hallada con la condición anterior
eq5 = eq4.subs({u: sln_u.rhs})
eq5

              4  -2⋅t
ẋ = 2⋅x - 18⋅ℯ ⋅ℯ    

In [256]:
# Se escriben las condiciones iniciales para la ecuación lineal anterior
ics2 = {x.subs(t,0) : 5}
ics2

{x(0): 5}

In [257]:
# Se resuelve la EDO lineal eq5 con las condiciones iniciales ics2
eqx = dsolve_system([eq5], ics=ics2)
sln_x = eqx[0][0].doit().simplify()
sln_x

    ⎛⎛        4⎞  4⋅t      4⎞  -2⋅t
    ⎝⎝10 - 9⋅ℯ ⎠⋅ℯ    + 9⋅ℯ ⎠⋅ℯ    
x = ───────────────────────────────
                   2               

In [258]:
print(latex(sln_x))

x{\left(t \right)} = \frac{\left(\left(10 - 9 e^{4}\right) e^{4 t} + 9 e^{4}\right) e^{- 2 t}}{2}


Finalmente, la variable de estado $x^*$ que minimiza el funcional objetivo $J$ es:

\begin{align*}
x{\left(t \right)} = \frac{\left(\left(10 - 9 e^{4}\right) e^{4 t} + 9 e^{4}\right) e^{- 2 t}}{2}
\end{align*}


## Problema 3

Obtener las funciones que verifiquen las condiciones necesarias de primer y segundo orden de máximo o mínimo local del siguiente problema:

\begin{align*}
J(x) & = \int_0^4(a\dot{x}^2 + bx + ct) \, dt \,, \qquad a,b,c \in \mathbb{R}, \,a\neq 0\\
\text{con :} & \quad x(0) = 2, \, x(4)=4
\end{align*}

Al igual que en el problema anterior, inicialmente se definen $t$ y $x$, además de las constantes $a,b$ y $c$. Además, se define el integrando del funcional $J$, $F = F(x,u,t)$. El problema no es de control en el dominio del tiempo, sino que se trata de las condiciones necesarias de optimalidad en un problema de control estático.

In [259]:
# Se define la variable independiente t
t = Symbol('t')

# Se define la trayectoria x(t)
x = Function('x')(t)

# Se declaran los símbolos a,b,c y las derivadas temporales de x
a, b, c = symbols('a, b, c')
dx = diff(x,t)
d2x = diff(dx, t)

# Definimos el integrando del funcional objetivo
F = a*dx**2 + b*x + c*t

# Se imprime F
F

   2            
a⋅ẋ  + b⋅x + c⋅t

Inicialmente se verifican las condiciones de primer orden (optimalidad), mediante el uso de la ecuación de Euler.

\begin{align*}
\frac{\partial F}{\partial x} - \frac{d}{dt} \left(\frac{\partial F}{\partial \dot{x}}\right) = 0
\end{align*}

In [260]:
# Se establece y enseña la ecuación de Euler luego de hacer todas las derivadas
eq_euler = Eq(diff(F,x) - diff(diff(F, dx), t), 0)
eq_euler

-2⋅a⋅ẍ + b = 0

Luego, se obtiene la EDO de segundo orden:

\begin{align}
-2a\ddot{x} + b = 0
\end{align}

con las condiciones de frontera:

\begin{align*}
x(0) = 2, \, x(4)=4
\end{align*}

Por el teorema de existencia y unicidad, este problema tiene solución única. Esta es hallada mediante método dsolve de Sympy.

In [261]:
# Primero establecemos las condiciones de contorno
jcs1 = {x.subs(t,0) : 2, x.subs(t,4) : 4}
jcs1

{x(0): 2, x(4): 4}

In [262]:
# Luego
eqx2 = dsolve_system([eq_euler], ics=jcs1)
sln_x2 = eqx2[0][0].doit().simplify()
sln_x2

               2      
    t       b⋅t    b⋅t
x = ─ + 2 + ──── - ───
    2       4⋅a     a 

Luego, la función

\begin{align}
x^*{\left(t \right)} = \frac{t}{2} + 2 + \frac{b t^{2}}{4 a} - \frac{b t}{a}
\end{align}

es la única $x \in \Omega$ que satisface la condición de primer orden. Falta analizar si esta función es un máximo o un mínimo para el problema 3.

Para ver esta diferenciación, usamos el criterio de Laguerre, o las condiciones de segundo orden, que indican que:

\begin{align}
\frac{∂^2F}{∂\dot{x}^2} >0 \quad \implies \quad x=x^*(t)\text{ es un mínimo local del problema}\\
\frac{∂^2F}{∂\dot{x}^2} >0 \quad \implies \quad x=x^*(t)\text{ es un máximo local del problema}
\end{align}

Entonces, derivando $F$ dos veces con respecto a la derivada de la coordenada generalizada $x$ se obtiene.

In [263]:
# Derivando dos veces F con respecto a dot{x}:
d2F = diff(diff(F,dx), dx)
d2F

2⋅a

Se obtuvo que $∀x \in \Omega$, $\frac{∂^2F}{∂\dot{x}^2} = 2a$. Luego, se obtiene este resultado también para $x^*$.

Entonces, según la condición de Laguerre, $x^*(t)$ es un máximo del problema 3 siempre que $a <0$. Por el contrario, $x^*(t)$ es un mínimo del problema 3 siempre que $a >0$.

## Problema 4
Calcular el valor de las variables de control, de estado y coestado para los que se verifica el principio del máximo de Pontryagin en el siguiente problema.

\begin{align*}
  \max J(x) & = \int_1^5 (ux-u^2-x^2) \, dt\\
  \text{sujeto a } & :\quad \dot{x} = x+u\\
  \text{con } & : \quad x(1) = 2
\end{align*}

Al igual que en el problema 2, se definen la variable independiente $t$, las funciones $x=x(t), u=u(t)$ y el integrando del funcional objetivo en forma de Lagrange $F = F(x,u,t) = ux-u^2-x^2$. Además, se crean variables para las derivadas de $x$ y se declara la función de restricción $f = f(x,u,t) = x+u$.

In [264]:
# Se declara la variable independiente t
t = Symbol('t')

# Se declaran x=x(t), u=u(t)
x = Function('x')(t)
u = Function('u')(t)

# Se define el integrando del funcional objetivo (forma de Lagrange)
F = u*x - u**2 - x**2

# Se crean variables para las derivadas de x y se define la restricción f
dx = diff(x, t)
d2x = diff(dx, t)
f = x + u

# Se imprime el integrando del funcional objetivo F
F

   2          2
- u  + u⋅x - x 

El siguiente paso es definir el hamiltoniano del sistema como:

\begin{align*}
H = H(x,u,λ,t) = F + λf
\end{align*}

donde $λ: \mathbb{R}^1 → \mathbb{R}$ es también una función de $t$.

In [265]:
# Se declara el multiplicador lambda del principio del máximo
lam = Function('lambda')(t)

# Se calcula el hamiltoniano como se enseña arriba
H = F + lam*f

# Se expone el hamiltoniano del sistema
H

             2          2
(u + x)⋅λ - u  + u⋅x - x 

Ahora, se emplea la condición $(1)$ del principio del máximo:

\begin{align*}
\dot{λ} = - \frac{\partial H}{∂ x}
\end{align*}

en conjunto con la condición inicial de la EDO anterior:

\begin{align*}
\lambda(5) = \frac{d}{dx}S(x^*(5)) = \frac{d}{dx}(0) = 0
\end{align*}

La condición final es nula, dado que el problema está escrito en forma de Lagrange.

In [266]:
# Se define la primera derivada temporal de lam
dlam = diff(lam, t)

# Se establece la ecuación diferencial de la condición (1)
neq1 = Eq(dlam, -diff(H,x))
neq1

λ̇ = -λ - u + 2⋅x

Aún no se puede resolver esta EDO, puesto que $\dot{λ}$ quedó en términos de $x$ y $u$, y para ello se procede con el resto del principio del máximo de Pontryagin.

A continuación, se debe usar la condición $(2)$ del principio del máximo. Para ello hay que calcular:

\begin{align*}
\max_u H(x^*,u,\lambda^*,t)
\end{align*}

donde $u$ se encuentra irrestricta.

Como se trata de una función univariada que se debe optimizar sin restricciones, se debe cumplir que:

\begin{align*}
\frac{∂H}{∂u} = 0, \qquad \frac{∂^2H}{∂u^2} < 0
\end{align*}

que son las condiciones de primer y segundo orden para hallar un mínimo global.

Se resuelve entonces:

\begin{align*}
  \max_u H = x + u + λ(1-u^2)
\end{align*}

In [267]:
# Se escribe la condición de primer orden para optimizar u
neq2 = Eq(diff(H,u), 0)
neq2

λ - 2⋅u + x = 0

In [268]:
# y se despeja u de la ecuación anterior
u_aux = solve([neq2], (u))
neq3 = Eq(u, u_aux[u])
neq3

    λ   x
u = ─ + ─
    2   2

Se obtuvo que

\begin{align}
u^*(t) =\frac{\lambda}{2} + \frac{x}{2}
\end{align}

es aquella función que verifica las condiciones de primer orden para optimizar el Hamiltoniano. Se debe verificar que $u$ maximice el hamiltoniano y no lo minimice.

Entonces, derivamos dos veces y aplicamos el criterio de la segunda derivada:

In [269]:
diff(diff(H,u),u)

-2

Como $\frac{∂^2H}{∂u^2} < 0$, $u$ maximiza el Hamiltoniano, y como esta función existe, el problema 3 aún es factible.

Ahora, se debe usar la condición $(3)$ del principio del máximo que corresponde a resolver la EDO contemplada en la restricción del problema:

\begin{align*}
\dot{x}^* = f(x^*, u^*,t) = x+u
\end{align*}

sujeta a la condición inicial $x(1)=2$.

In [270]:
# Usamos la restricción del problema 3
neq4 = Eq(dx, f)
neq4

ẋ = u + x

In [271]:
# Se sustituye la u hallada en el numeral anterior
neq5 = neq4.subs(u_aux)
neq5

    λ   3⋅x
ẋ = ─ + ───
    2    2 

In [272]:
# y sustituimos u en la ecuación para \dot{lam}
neq6 = neq1.subs(u_aux)
neq6

      3⋅λ   3⋅x
λ̇ = - ─── + ───
       2     2 

En este punto, a partir de las condiciones $(1)$ y $(3)$ del principio del máximo y habiendo reemplazado $u^*(t) =\frac{\lambda}{2} + \frac{x}{2}
$ en ambas, se obtiene el sistema de EDOs de primer orden:

\begin{gather*}
\frac{d}{d t} \lambda{\left(t \right)} = - \frac{3 \lambda{\left(t \right)}}{2} + \frac{3 x{\left(t \right)}}{2}\\
\frac{d}{d t} x{\left(t \right)} = \frac{\lambda{\left(t \right)}}{2} + \frac{3
x{\left(t \right)}}{2}
\end{gather*}

Sujeto a las condiciones de contorno:

\begin{align}
x(1) = 2, \quad λ(5) = 0
\end{align}

Se resuelve este sistema de EDOs mediante el comando dsolve de Sympy.

In [273]:
# Se escriben las condiciones de contorno como un diccionario
kcs={x.subs(t,1): 2, lam.subs(t,5): 0}
kcs

{λ(5): 0, x(1): 2}

In [243]:
# Se soluciona el sistema de EDOs con las condiciones de frontera anteriores
slnn = dsolve_system([neq5, neq6], ics=kcs)
sln_x3 = slnn[0][0].doit().simplify()
sln_lam3 = slnn[0][1].doit().simplify()

# Se enseña la solución para x
display(sln_x3)

      ⎛⎛          8⋅√3      8⋅√3⎞  2⋅√3⋅t   ⎛             8⋅√3⎞  10⋅√3⎞  -√3⋅(
    2⋅⎝⎝1 + 4⋅√3⋅ℯ     + 7⋅ℯ    ⎠⋅ℯ       + ⎝-4⋅√3 + 7 + ℯ    ⎠⋅ℯ     ⎠⋅ℯ     
x = ──────────────────────────────────────────────────────────────────────────
                                         8⋅√3    16⋅√3                        
                                 1 + 14⋅ℯ     + ℯ                             

t + 1)
      
──────
      
      

In [274]:
# Se enseña la solución para lambda
display(sln_lam3)

      ⎛⎛               8⋅√3         8⋅√3⎞  2⋅√3⋅t   ⎛       ⎛        8⋅√3    1
    2⋅⎝⎝-3 + 2⋅√3 + 3⋅ℯ     + 2⋅√3⋅ℯ    ⎠⋅ℯ       + ⎝- 2⋅√3⋅⎝1 + 14⋅ℯ     + ℯ 
λ = ──────────────────────────────────────────────────────────────────────────
                                                                        8⋅√3  
                                                                1 + 14⋅ℯ     +

6⋅√3⎞      16⋅√3             8⋅√3          8⋅√3⎞  2⋅√3⎞  -√3⋅(t + 1)
    ⎠ - 3⋅ℯ      + 2⋅√3 + 3⋅ℯ     + 26⋅√3⋅ℯ    ⎠⋅ℯ    ⎠⋅ℯ           
────────────────────────────────────────────────────────────────────
  16⋅√3                                                             
 ℯ                                                                  

Finalmente, el control óptimo $u^*$ se halla como:

\begin{align}
u^*(t) = \frac{1}{2} [x^*(t) + λ^*(t)]
\end{align}

reemplazando estas dos expresiones obtenidas para la trayectoria y el multiplicador óptimos se obtiene:

In [276]:
neq3.subs({x: sln_x3.rhs, lam: sln_lam3.rhs})

    ⎛⎛          8⋅√3      8⋅√3⎞  2⋅√3⋅t   ⎛             8⋅√3⎞  10⋅√3⎞  -√3⋅(t 
    ⎝⎝1 + 4⋅√3⋅ℯ     + 7⋅ℯ    ⎠⋅ℯ       + ⎝-4⋅√3 + 7 + ℯ    ⎠⋅ℯ     ⎠⋅ℯ       
u = ──────────────────────────────────────────────────────────────────────────
                                        8⋅√3    16⋅√3                         
                                1 + 14⋅ℯ     + ℯ                              

+ 1)   ⎛⎛               8⋅√3         8⋅√3⎞  2⋅√3⋅t   ⎛       ⎛        8⋅√3    
       ⎝⎝-3 + 2⋅√3 + 3⋅ℯ     + 2⋅√3⋅ℯ    ⎠⋅ℯ       + ⎝- 2⋅√3⋅⎝1 + 14⋅ℯ     + ℯ
──── + ───────────────────────────────────────────────────────────────────────
                                                                          8⋅√3
                                                                  1 + 14⋅ℯ    

16⋅√3⎞      16⋅√3             8⋅√3          8⋅√3⎞  2⋅√3⎞  -√3⋅(t + 1)
     ⎠ - 3⋅ℯ      + 2⋅√3 + 3⋅ℯ     + 26⋅√3⋅ℯ    ⎠⋅ℯ    ⎠⋅ℯ           
────────────────────────────────────────────────────────────────────

que es el control óptimo de la planta.